In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

use this---------

## load data

In [ ]:
! wget "https://he-public-data.s3.ap-southeast-1.amazonaws.com/shell_dataset.zip"
! unzip shell_dataset.zip
! unzip -q dataset/train.zip
! unzip -q dataset/test.zip

## imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
import sys
import os
from joblib import load,dump
import warnings
warnings.filterwarnings('ignore')

random_seed = 42

In [ ]:
train_data = pd.read_csv('./train/train.csv')
train_data

## clean data

replace unwanted values and interpolate

In [ ]:
train_data['Total Cloud Cover [%]'].replace(-7999,np.nan,inplace = True)
train_data['Total Cloud Cover [%]'].replace(-6999,np.nan,inplace = True)
train_data['Total Cloud Cover [%]'].interpolate(limit = 10,limit_direction = 'both',inplace = True)  # possiblity of data leak

In [ ]:
train_data['t_30'] = train_data.groupby('DATE (MM/DD)')['Total Cloud Cover [%]'].shift(periods = -30,fill_value = -1)
train_data['t_60'] = train_data.groupby('DATE (MM/DD)')['Total Cloud Cover [%]'].shift(periods = -60,fill_value = -1)
train_data['t_90'] = train_data.groupby('DATE (MM/DD)')['Total Cloud Cover [%]'].shift(periods = -90,fill_value = -1)
train_data['t_120'] = train_data.groupby('DATE (MM/DD)')['Total Cloud Cover [%]'].shift(periods = -120,fill_value = -1)
train_data

In [ ]:
cond = (train_data['t_30'] == -1) | (train_data['t_90'] == -1) | (train_data['t_60'] == -1) | (train_data['t_120'] == -1)
train_data[cond].shape

In [ ]:
# train[~cond]['t_30'].value_counts()
# samples that are to be 
req_samples = train_data[cond].sample(frac = 0.02,random_state = random_seed)
req_samples.shape

In [ ]:
not_req_samples = train_data[cond].drop(req_samples.index)
not_req_samples.shape                                   # this is some lame code.................

In [ ]:
train_data.drop(not_req_samples.index,inplace=True)
train_data.value_counts()

In [ ]:
train_data.columns

In [ ]:
train_data.drop(columns={
    'DATE (MM/DD)',
    'MST',
    'Direct sNIP [W/m^2]',                # this feature is highly correlated with cmp22
    'Tower Wet Bulb Temp [deg C]',        # highly correlated with other temperature readings
    'Snow Depth [cm]' 
},inplace = True)
train_data.head()

## cross_validate

In [ ]:
X_30,Y_30 = train_data.iloc[:,:-4],train_data['t_30']
X_60,Y_60 = train_data.iloc[:,:-3],train_data['t_60']
X_90,Y_90 = train_data.iloc[:,:-2],train_data['t_90']
X_120,Y_120 = train_data.iloc[:,:-1],train_data['t_120']

In [ ]:
print(X_30.shape,Y_30.shape)
print(X_60.shape,Y_60.shape)
print(X_90.shape,Y_90.shape)

### model

In [ ]:
params = {
    'n_estimators':[x for x in range(900,1100,100)],
    'max_depth':[x for x in range(8,10)],
#     'bootstrap':[True,False]
#     'min_samples_split':[x for x in range(50,500,20)],
#     'max_features':['auto', 'sqrt', 'log2']
}

In [ ]:
rf_30 = RandomForestRegressor(random_state = random_seed,n_jobs=-1)

In [ ]:
%%time
rand_search_30 = RandomizedSearchCV(estimator= rf_30,param_distributions =params,cv = 2,random_state = random_seed)
rand_search_30.fit(X_30,Y_30)
print(rand_search_30.best_params_)

# rand_search_60 = RandomizedSearchCV(estimator= rf_60,param_distributions=params,cv = 2,random_state = random_seed,n_iter = 10)
# rand_search_60.fit(X_60,Y_60)
# print(rand_search_30.best_params_)

# rand_search_90 = RandomizedSearchCV(estimator= rf_90,param_distributions=params,1v = 2,random_state = random_seed,n_iter = 10)
# rand_search_90.fit(X_90,Y_901
# print(rand_search_30.best_params_)

# rand_search_120 = RandomizedSearchCV(estimator= rf_120,param_distributions=params,cv = 2,random_state = random_seed,n_iter = 10)
# rand_search_120.fit(X_120,Y_120)
# print(rand_search_30.best_params_)


In [ ]:
rand_search_30

In [ ]:
rf_30 = RandomForestRegressor(max_depth = 8,n_estimators = 900,random_state= random_seed,n_jobs=-1)

In [ ]:
rf_60 = RandomForestRegressor(max_depth = 8,n_estimators = 900,random_state= random_seed,n_jobs=-1)
rf_90 = RandomForestRegressor(max_depth = 8,n_estimators = 900,random_state= random_seed,n_jobs=-1)
rf_120 = RandomForestRegressor(max_depth = 8,n_estimators = 900,random_state= random_seed,n_jobs=-1)

In [ ]:
rf_30.fit(X_30,Y_30)
print('train score:',rf_30.score(X_30,Y_30))

In [ ]:
%%time
rf_60.fit(X_60,Y_60)
print('train score:',rf_60.score(X_60,Y_60))

rf_90.fit(X_90,Y_90)
print('train score:',rf_90.score(X_90,Y_90))

rf_120.fit(X_120,Y_120)
print('train score:',rf_120.score(X_120,Y_120))

In [ ]:
import joblib
# joblib.dump(rand_search_30,'rand_search_30.joblib')
joblib.dump(rf_30,'rf_30.joblib')

In [ ]:
joblib.dump(rf_60,'rf_60.joblib')
joblib.dump(rf_90,'rf_90.joblib')
joblib.dump(rf_120,'rf_120.joblib')

In [ ]:

# X_train_30,X_test_30,Y_train_30,Y_test_30 = train_test_split(train_d[['Total Cloud Cover [%]','Peak Wind Speed @ 6ft [m/s]','Avg Wind Direction @ 6ft [deg from N]']],train_d['t_30'],
#                                                  random_state = 42,
#                                                  train_size = 0.5)
# X_train_60,X_test_60,Y_train_60,Y_test_60 = train_test_split(train_d[['Total Cloud Cover [%]','Peak Wind Speed @ 6ft [m/s]','Avg Wind Direction @ 6ft [deg from N]','t_30']],train_d['t_60'],
#                                                  random_state = 42,
#                                                  train_size = 0.5)
# X_train_90,X_test_90,Y_train_90,Y_test_90 = train_test_split(train_d[['Total Cloud Cover [%]','Peak Wind Speed @ 6ft [m/s]','Avg Wind Direction @ 6ft [deg from N]','t_30','t_60']],train_d['t_90'],
#                                                  random_state = 42,
#                                                  train_size = 0.5)
# X_train_120,X_test_120,Y_train_120,Y_test_120 = train_test_split(train_d[['Total Cloud Cover [%]','Peak Wind Speed @ 6ft [m/s]','Avg Wind Direction @ 6ft [deg from N]','t_30','t_60','t_90']],train_d['t_120'],
#                                                  random_state = 42,
#                                                  train_size = 0.5)


In [ ]:
# clf_30 = SVR(max_iter = 1000_000)
# clf_30.fit(X_train_30,Y_train_30)
# print(f"clf_30 train score {clf_30.score(X_train_30,Y_train_30)}")
# y_pred_clf_30 = clf_30.predict(X_test_30)
# print(f"clf_30 test mse {mse(Y_test_30,y_pred_clf_30)}")

# clf_60 = SVR(max_iter = 1000_000)
# clf_60.fit(X_train_60,Y_train_60)
# print(f"clf_60 test score {clf_60.score(X_test_60,Y_test_60)}")
# y_pred_clf_60 = clf_60.predict(X_test_60)
# print(f"clf_60 test mse {mse(Y_test_60,y_pred_clf_60)}")

# clf_90 = SVR(max_iter = 1000_000)     
# clf_90.fit(X_train_90,Y_train_90)
# y_pred_clf_90 = clf_90.predict(X_test_90)
# print(f"clf_90 test mse {mse(Y_test_90,y_pred_clf_90)}")

# clf_120 = SVR(max_iter = 1000_000)
# clf_120.fit(X_train_120,Y_train_120)
# print(f"clf_120 test score {clf_120.score(X_test_120,Y_test_120)}")
# y_pred_clf_120 = clf_120.predict(X_test_120)
# print(f"clf_120 test mse {mse(Y_test_120,y_pred_clf_120)}")



In [ ]:
# clf_30 = RandomForestRegressor()
# clf_30.fit(X_train_30,Y_train_30)
# # print(f"clf_30 test score {clf_30.score(X_train_30,Y_train_30)}")
# y_pred_clf_30 = clf_30.predict(X_test_30)
# print(f"clf_30 test mse {mse(Y_test_30,y_pred_clf_30)}")

# clf_60 = RandomForestRegressor()
# clf_60.fit(X_train_60,Y_train_60)
# print(f"clf_60 test score {clf_60.score(X_test_60,Y_test_60)}")
# y_pred_clf_60 = clf_60.predict(X_test_60)
# print(f"clf_60 test mse {mse(Y_test_60,y_pred_clf_60)}")

# clf_90 = RandomForestRegressor()     
# clf_90.fit(X_train_90,Y_train_90)
# y_pred_clf_90 = clf_90.predict(X_test_90)
# print(f"clf_90 test mse {mse(Y_test_90,y_pred_clf_90)}")

# clf_120 = RandomForestRegressor()
# clf_120.fit(X_train_120,Y_train_120)
# # print(f"clf_120 test score {clf_120.score(X_test_120,Y_test_120)}")
# y_pred_clf_120 = clf_120.predict(X_test_120)
# print(f"clf_120 test mse {mse(Y_test_120,y_pred_clf_120)}")

# FINAL PREDICTIONS

In [ ]:
rf_30 = load('../input/shell-ai-randomforest-regressors/rf_30 (2).joblib')
rf_60 = load('../input/shell-ai-randomforest-regressors/rf_60.joblib')
rf_90 = load('../input/shell-ai-randomforest-regressors/rf_90.joblib')
rf_120 = load('../input/shell-ai-randomforest-regressors/rf_120.joblib')

In [ ]:
print(rf_30.feature_importances_)
print(rf_30.n_outputs_)
print(rf_60.feature_importances_)

In [ ]:
def preprocess(df,mode = 'test'):
    df['Total Cloud Cover [%]'].replace(-7999,np.nan,inplace = True)
    df['Total Cloud Cover [%]'].replace(-6999,np.nan,inplace = True)
    df['Total Cloud Cover [%]'].interpolate(limit = 10,limit_direction = 'both',inplace = True)
    if mode == 'train':
        # transform wind speed
#         pt_wind_speed = PowerTransformer()
#         pt_wind_speed.fit(df[['Peak Wind Speed @ 6ft [m/s]']])
#         df['Peak Wind Speed @ 6ft [m/s]'] = pt_wind_speed.transform(df[['Peak Wind Speed @ 6ft [m/s]']])
#         # transform wind dir
#         pt_wind_dir= PowerTransformer()
#         pt_wind_dir.fit(df[['Avg Wind Direction @ 6ft [deg from N]']])
#         df.loc[:,('Avg Wind Direction @ 6ft [deg from N]')] = pt_wind_dir.transform(df[['Avg Wind Direction @ 6ft [deg from N]']])
        df['t_30'] = df.groupby('DATE (MM/DD)')['Total Cloud Cover [%]'].shift(periods = -30,fill_value = -1)
        df['t_60'] = df.groupby('DATE (MM/DD)')['Total Cloud Cover [%]'].shift(periods = -60,fill_value = -1)
        df['t_90'] = df.groupby('DATE (MM/DD)')['Total Cloud Cover [%]'].shift(periods = -90,fill_value = -1)
        df['t_120'] = df.groupby('DATE (MM/DD)')['Total Cloud Cover [%]'].shift(periods = -120,fill_value = -1)
        df
    if mode == 'test':
        df.drop([
            'Direct sNIP [W/m^2]',
            'Time [Mins]',
            'Tower Wet Bulb Temp [deg C]',
            'Snow Depth [cm]'
        ],axis=1,inplace=True)
        
    return df.iloc[-1,]
        

In [ ]:
path = os.getcwd()
name = "weather_data.csv"
test = pd.read_csv('./dataset/test.csv')

file_count = 0
for root, dirs, files in os.walk(path):
    if name in files:
        # take in test data
        req_path = (os.path.join(root, name))
        wd = pd.read_csv(req_path)
        scenario_set = int(req_path.split('/')[-2])
        
        # preprocessing test data
        last_sample = preprocess(wd)
        
        # predicting test samples
        pred_30 = rf_30.predict(last_sample.to_numpy().reshape(1,-1))
        last_sample['t_30'] = pred_30[0]
        pred_60 = rf_60.predict(last_sample.to_numpy().reshape(1,-1))
        last_sample['t_60'] = pred_60[0]
        pred_90 = rf_90.predict(last_sample.to_numpy().reshape(1,-1))
        last_sample['t_90'] = pred_90[0]
        pred_120 = rf_120.predict(last_sample.to_numpy().reshape(1,-1))
        
        # fill in test data using above predictions
        test.iloc[scenario_set-1,test.columns.get_indexer(['30_min_horizon'])] = np.round(pred_30[0])
        test.iloc[scenario_set-1,test.columns.get_indexer(['60_min_horizon'])] = np.round(pred_60[0])
        test.iloc[scenario_set-1,test.columns.get_indexer(['90_min_horizon'])] = np.round(pred_90[0])
        test.iloc[scenario_set-1,test.columns.get_indexer(['120_min_horizon'])] = np.round(pred_120[0])

        file_count += 1
        print(file_count)

In [ ]:
type(test['30_min_horizon'][0])

In [ ]:
test.head()

In [ ]:
test = test.applymap(int)
test

In [ ]:
test.isna().sum()

In [ ]:
test.to_csv('rf_tuned_correct_preds.csv',index=False)
test
# my best preds---> 85% acc

In [ ]:
submission = pd.read_csv('./dataset/sample_submission.csv')
submission